# 🩸 Final Girl  
## *Episode 1: Scream Queens*

It always starts the same.  
A promising lineup. A few fan favorites.  
But somewhere in the details… a death sentence.

Let’s spot the tropes to decide who gets a sequel. 
*The metadata always knows who dies first.*


In [1]:
import pandas as pd

# Load the TMDb dataset
df = pd.read_csv("../data/TMDB.csv")

# Preview the first few rows
df.head()

,id,name,number_of_seasons,number_of_episodes,original_language,vote_count,vote_average,overview,adult,backdrop_path,...,tagline,genres,created_by,languages,networks,origin_country,spoken_languages,production_companies,production_countries,episode_run_time
0,1399,Game of Thrones,8,73,en,21857,8.442,Seven noble families fight for control of the ...,False,/2OMB0ynKlyIenMJWI2Dy9IWT4c.jpg,...,Winter Is Coming,"Sci-Fi & Fantasy, Drama, Action & Adventure","David Benioff, D.B. Weiss",en,HBO,US,English,"Revolution Sun Studios, Television 360, Genera...","United Kingdom, United States of America",0
1,71446,Money Heist,3,41,es,17836,8.257,"To carry out the biggest heist in history, a m...",False,/gFZriCkpJYsApPZEF3jhxL4yLzG.jpg,...,The perfect robbery.,"Crime, Drama",Álex Pina,es,"Netflix, Antena 3",ES,Español,Vancouver Media,Spain,70
2,66732,Stranger Things,4,34,en,16161,8.624,"When a young boy vanishes, a small town uncove...",False,/2MaumbgBlW1NoPo3ZJO38A6v7OS.jpg,...,Every ending has a beginning.,"Drama, Sci-Fi & Fantasy, Mystery","Matt Duffer, Ross Duffer",en,Netflix,US,English,"21 Laps Entertainment, Monkey Massacre Product...",United States of America,0
3,1402,The Walking Dead,11,177,en,15432,8.121,Sheriff's deputy Rick Grimes awakens from a co...,False,/x4salpjB11umlUOltfNvSSrjSXm.jpg,...,Fight the dead. Fear the living.,"Action & Adventure, Drama, Sci-Fi & Fantasy",Frank Darabont,en,AMC,US,English,"AMC Studios, Circle of Confusion, Valhalla Mot...",United States of America,42
4,63174,Lucifer,6,93,en,13870,8.486,"Bored and unhappy as the Lord of Hell, Lucifer...",False,/aDBRtunw49UF4XmqfyNuD9nlYIu.jpg,...,It's good to be bad.,"Crime, Sci-Fi & Fantasy",Tom Kapinos,en,"FOX, Netflix",US,English,"Warner Bros. Television, DC Entertainment, Jer...",United States of America,45


In [2]:
# Get a quick summary of the dataset
df.info()

# Check for missing values
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168639 entries, 0 to 168638
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    168639 non-null  int64  
 1   name                  168634 non-null  object 
 2   number_of_seasons     168639 non-null  int64  
 3   number_of_episodes    168639 non-null  int64  
 4   original_language     168639 non-null  object 
 5   vote_count            168639 non-null  int64  
 6   vote_average          168639 non-null  float64
 7   overview              93333 non-null   object 
 8   adult                 168639 non-null  bool   
 9   backdrop_path         77780 non-null   object 
 10  first_air_date        136903 non-null  object 
 11  last_air_date         138735 non-null  object 
 12  homepage              50998 non-null   object 
 13  in_production         168639 non-null  bool   
 14  original_name         168634 non-null  object 
 15  

id                           0
name                         5
number_of_seasons            0
number_of_episodes           0
original_language            0
vote_count                   0
vote_average                 0
overview                 75306
adult                        0
backdrop_path            90859
first_air_date           31736
last_air_date            29904
homepage                117641
in_production                0
original_name                5
popularity                   0
poster_path              59902
type                         0
status                       0
tagline                 163309
genres                   68926
created_by              132143
languages                58589
networks                 71050
origin_country           31030
spoken_languages         59359
production_companies    109297
production_countries     91128
episode_run_time             0
dtype: int64

In [3]:
# Check unique values in survival-related columns
df["status"].value_counts()
df["in_production"].value_counts()

in_production
False    97866
True     70773
Name: count, dtype: int64

In [4]:
# Check the unique values in the 'status' column
df["status"].value_counts()

status
Ended               93232
Returning Series    67507
Canceled             4634
In Production        2380
Planned               650
Pilot                 236
Name: count, dtype: int64

In [5]:
# Label the survivors
df["is_final_girl"] = df["status"].isin(["Returning Series", "In Production"])

# Preview the survival split
df["is_final_girl"].value_counts()


is_final_girl
False    98752
True     69887
Name: count, dtype: int64

In [6]:
# Compare average values for survivors vs. slashed
df.groupby("is_final_girl")[["number_of_seasons", "vote_average", "popularity"]].mean()


,number_of_seasons,vote_average,popularity
is_final_girl,,,
False,1.249605,2.750512,5.960951
True,1.970839,1.745081,5.771995


In [7]:
# Create a flag for whether a show has an overview
df["has_overview"] = df["overview"].notnull()

# Compare vote_count and overview presence by survival
df.groupby("is_final_girl")[["vote_count", "has_overview"]].mean()


,vote_count,has_overview
is_final_girl,,
False,17.272845,0.609952
True,7.698470,0.473607


In [8]:
import ast

# Function to extract genre names
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [g["name"] for g in genres if "name" in g]
    except:
        return []

# Apply to the dataset
df["genre_list"] = df["genres"].apply(extract_genres)

# Preview a few entries
df[["genres", "genre_list"]].sample(5, random_state=42)


,genres,genre_list
135033,Documentary,[]
75653,NaN,[]
156984,NaN,[]
112486,"Crime, Drama",[]
12844,"Animation, Comedy, Sci-Fi & Fantasy, Action & ...",[]


In [9]:
# Function to split comma-separated genre strings
def split_genres(genre_str):
    if pd.isnull(genre_str):
        return []
    return [g.strip() for g in genre_str.split(",")]

# Apply to the dataset
df["genre_list"] = df["genres"].apply(split_genres)

# Preview a few entries
df[["genres", "genre_list"]].sample(5, random_state=42)


,genres,genre_list
135033,Documentary,[Documentary]
75653,NaN,[]
156984,NaN,[]
112486,"Crime, Drama","[Crime, Drama]"
12844,"Animation, Comedy, Sci-Fi & Fantasy, Action & ...","[Animation, Comedy, Sci-Fi & Fantasy, Action &..."


In [10]:
from collections import Counter

# Flatten all genre lists into one big list
all_genres = [genre for sublist in df["genre_list"] for genre in sublist]

# Count genre frequencies
genre_counts = Counter(all_genres)

# Preview top 15 genres
genre_counts.most_common(15)


[('Drama', 33668),
 ('Comedy', 23229),
 ('Documentary', 22031),
 ('Animation', 11971),
 ('Reality', 11018),
 ('Action & Adventure', 7266),
 ('Crime', 6844),
 ('Family', 6474),
 ('Sci-Fi & Fantasy', 6369),
 ('Mystery', 5264),
 ('Kids', 4536),
 ('Talk', 3229),
 ('Soap', 2042),
 ('War & Politics', 1767),
 ('News', 1627)]

In [11]:
# Create binary flags for top genres
top_genres = ["Drama", "Comedy", "Documentary", "Animation", "Reality"]

for genre in top_genres:
    df[f"has_{genre.lower()}"] = df["genre_list"].apply(lambda x: genre in x)

# Compare survival rates by genre
df.groupby("is_final_girl")[[f"has_{g.lower()}" for g in top_genres]].mean()


,has_drama,has_comedy,has_documentary,has_animation,has_reality
is_final_girl,,,,,
False,0.253008,0.168948,0.121324,0.088626,0.040283
True,0.123900,0.093580,0.143732,0.046031,0.100720


In [12]:
# Add more genre flags
more_genres = ["Crime", "Family", "Sci-Fi & Fantasy"]
all_genres_to_check = ["Drama", "Comedy", "Documentary", "Animation", "Reality"] + more_genres

for genre in all_genres_to_check:
    df[f"has_{genre.lower().replace(' & ', '_and_').replace(' ', '_')}"] = df["genre_list"].apply(lambda x: genre in x)

# Compare survival rates by genre
df.groupby("is_final_girl")[[f"has_{g.lower().replace(' & ', '_and_').replace(' ', '_')}" for g in all_genres_to_check]].mean()


,has_drama,has_comedy,has_documentary,has_animation,has_reality,has_crime,has_family,has_sci-fi_and_fantasy
is_final_girl,,,,,,,,
False,0.253008,0.168948,0.121324,0.088626,0.040283,0.045579,0.042986,0.050662
True,0.123900,0.093580,0.143732,0.046031,0.100720,0.033497,0.031866,0.019532


In [13]:
# Function to split comma-separated network strings
def split_networks(network_str):
    if pd.isnull(network_str):
        return []
    return [n.strip() for n in network_str.split(",")]

# Apply to the dataset
df["network_list"] = df["networks"].apply(split_networks)

# Preview a few entries
df[["networks", "network_list"]].sample(5, random_state=42)


,networks,network_list
135033,ABC TV,[ABC TV]
75653,NaN,[]
156984,NaN,[]
112486,Sveriges Radio & TV,[Sveriges Radio & TV]
12844,NaN,[]


In [14]:
# Count network frequencies
network_counts = df["networks"].value_counts().head(20)

# Preview top 20 networks
network_counts


networks
BBC One           2103
YouTube           1859
Netflix           1775
ITV1              1767
BBC Two           1626
ABC               1475
NBC               1419
TVB Jade          1365
CBS               1329
Channel 4         1254
ZDF               1065
Fuji TV            963
CBC Television     918
ARD                836
Prime Video        809
TV Tokyo           804
TBS                780
SBS                725
FOX                689
TV Globo           682
Name: count, dtype: int64

In [15]:
# Select top networks to analyze
top_networks = ["Netflix", "YouTube", "BBC One", "ABC", "Prime Video"]

# Create binary flags
for network in top_networks:
    df[f"is_{network.lower().replace(' ', '_')}"] = df["network_list"].apply(lambda x: network in x)

# Compare survival rates by network
df.groupby("is_final_girl")[[f"is_{n.lower().replace(' ', '_')}" for n in top_networks]].mean()


,is_netflix,is_youtube,is_bbc_one,is_abc,is_prime_video
is_final_girl,,,,,
False,0.012212,0.010876,0.018582,0.014835,0.003909
True,0.010775,0.020376,0.006396,0.001846,0.007913


In [16]:
# Add legacy networks
legacy_networks = ["NBC", "CBS", "FOX"]

for network in legacy_networks:
    df[f"is_{network.lower()}"] = df["networks"].apply(lambda x: network in str(x) if pd.notnull(x) else False)

# Compare survival rates by network
df.groupby("is_final_girl")[[f"is_{n.lower()}" for n in legacy_networks]].mean()


,is_nbc,is_cbs,is_fox
is_final_girl,,,
False,0.016212,0.013498,0.006967
True,0.002003,0.001746,0.001874


In [17]:
# Function to split comma-separated company strings
def split_companies(company_str):
    if pd.isnull(company_str):
        return []
    return [c.strip() for c in company_str.split(",")]

# Apply to the dataset
df["company_list"] = df["production_companies"].apply(split_companies)

# Preview a few entries
df[["production_companies", "company_list"]].sample(5, random_state=42)


,production_companies,company_list
135033,Northern Pictures,[Northern Pictures]
75653,NaN,[]
156984,NaN,[]
112486,Sveriges Radio (SR),[Sveriges Radio (SR)]
12844,"Anime International Company, Pioneer LDC","[Anime International Company, Pioneer LDC]"


In [18]:
import re

# Function to split production companies more reliably
def clean_companies(company_str):
    if pd.isnull(company_str):
        return []
    # Split on commas not inside parentheses
    return [c.strip() for c in re.split(r',(?![^()]*\))', company_str)]

# Apply to the dataset
df["company_list"] = df["production_companies"].apply(clean_companies)

# Preview a few entries
df[["production_companies", "company_list"]].sample(5, random_state=42)


,production_companies,company_list
135033,Northern Pictures,[Northern Pictures]
75653,NaN,[]
156984,NaN,[]
112486,Sveriges Radio (SR),[Sveriges Radio (SR)]
12844,"Anime International Company, Pioneer LDC","[Anime International Company, Pioneer LDC]"


In [19]:
from collections import Counter

# Flatten all company lists
all_companies = [company for sublist in df["company_list"] for company in sublist]

# Count frequencies
company_counts = Counter(all_companies)

# Preview top 20
company_counts.most_common(20)


[('TVB', 1388),
 ('BBC', 874),
 ('Estúdios Globo', 635),
 ('NHK', 586),
 ('Česká televize', 398),
 ('Warner Bros. Television', 382),
 ('Universal Television', 349),
 ('Televisa', 348),
 ('DR TV', 343),
 ('TV 2', 339),
 ('ARTE', 323),
 ('ZDF', 320),
 ('Fuji Television Network', 308),
 ('GMA Entertainment Group', 293),
 ('TBS', 288),
 ('RAI', 287),
 ('TV Tokyo', 280),
 ('SVT', 273),
 ('ATV Enterprises Limited', 272),
 ('Amazon Studios', 269)]

In [20]:
# Select top studios to analyze
top_companies = ["TVB", "BBC", "Warner Bros. Television", "Universal Television", "Amazon Studios"]

# Create binary flags
for company in top_companies:
    df[f"has_{company.lower().replace(' ', '_').replace('.', '').replace('&', 'and')}"] = df["company_list"].apply(lambda x: company in x)

# Compare survival rates by company
df.groupby("is_final_girl")[[f"has_{c.lower().replace(' ', '_').replace('.', '').replace('&', 'and')}" for c in top_companies]].mean()


,has_tvb,has_bbc,has_warner_bros_television,has_universal_television,has_amazon_studios
is_final_girl,,,,,
False,0.013752,0.007362,0.003453,0.002997,0.001337
True,0.000429,0.002103,0.000587,0.000758,0.001932


In [21]:
# Vote-based features
df["is_highly_rated"] = df["vote_average"] > 7
df["is_popular"] = df["vote_count"] > 100
df["is_high_quality"] = df["is_highly_rated"] & df["is_popular"]

# Preview survival rates
df.groupby("is_final_girl")[["is_highly_rated", "is_popular", "is_high_quality"]].mean()


,is_highly_rated,is_popular,is_high_quality
is_final_girl,,,
False,0.189859,0.025437,0.021529
True,0.125088,0.007140,0.006439


In [22]:
# Basic stats
df["popularity"].describe()


count    168639.000000
mean          5.882644
std          42.023216
min           0.000000
25%           0.600000
50%           0.857000
75%           2.431500
max        3707.008000
Name: popularity, dtype: float64

In [23]:
# Popularity tiers
df["is_obscure"] = df["popularity"] <= 1.0
df["is_moderately_popular"] = (df["popularity"] > 1.0) & (df["popularity"] <= 10.0)
df["is_hyped"] = df["popularity"] > 10.0

# Compare survival rates
df.groupby("is_final_girl")[["is_obscure", "is_moderately_popular", "is_hyped"]].mean()


,is_obscure,is_moderately_popular,is_hyped
is_final_girl,,,
False,0.493367,0.405359,0.101274
True,0.606780,0.327429,0.065792


In [24]:
# Basic stats
df["number_of_episodes"].describe()


count    168639.000000
mean         24.465082
std         134.799622
min           0.000000
25%           1.000000
50%           6.000000
75%          20.000000
max       20839.000000
Name: number_of_episodes, dtype: float64

In [25]:
# Episode count tiers
df["is_miniseries"] = df["number_of_episodes"] <= 6
df["is_mid_length"] = (df["number_of_episodes"] > 6) & (df["number_of_episodes"] <= 20)
df["is_long_running"] = df["number_of_episodes"] > 20

# Compare survival rates
df.groupby("is_final_girl")[["is_miniseries", "is_mid_length", "is_long_running"]].mean()


,is_miniseries,is_mid_length,is_long_running
is_final_girl,,,
False,0.493995,0.264622,0.241382
True,0.537081,0.246641,0.216278


In [26]:
# Select all boolean and numeric columns
correlation_features = df.select_dtypes(include=["bool", "int64", "float64"])

# Check which columns are included
print(correlation_features.columns.tolist())


['id', 'number_of_seasons', 'number_of_episodes', 'vote_count', 'vote_average', 'adult', 'in_production', 'popularity', 'episode_run_time', 'is_final_girl', 'has_overview', 'has_drama', 'has_comedy', 'has_documentary', 'has_animation', 'has_reality', 'has_crime', 'has_family', 'has_sci-fi_and_fantasy', 'is_netflix', 'is_youtube', 'is_bbc_one', 'is_abc', 'is_prime_video', 'is_nbc', 'is_cbs', 'is_fox', 'has_tvb', 'has_bbc', 'has_warner_bros_television', 'has_universal_television', 'has_amazon_studios', 'is_highly_rated', 'is_popular', 'is_high_quality', 'is_obscure', 'is_moderately_popular', 'is_hyped', 'is_miniseries', 'is_mid_length', 'is_long_running']


In [27]:
# Compute correlation matrix
correlation_matrix = df[[
    'is_final_girl',
    'number_of_seasons', 'number_of_episodes', 'vote_count', 'vote_average', 'popularity',
    'has_drama', 'has_comedy', 'has_documentary', 'has_animation', 'has_reality', 'has_crime', 'has_family', 'has_sci-fi_and_fantasy',
    'is_netflix', 'is_youtube', 'is_bbc_one', 'is_abc', 'is_prime_video', 'is_nbc', 'is_cbs', 'is_fox',
    'has_tvb', 'has_bbc', 'has_warner_bros_television', 'has_universal_television', 'has_amazon_studios',
    'is_highly_rated', 'is_popular', 'is_high_quality',
    'is_obscure', 'is_moderately_popular', 'is_hyped',
    'is_miniseries', 'is_mid_length', 'is_long_running'
]].corr()

# Sort by correlation with survival
correlation_matrix["is_final_girl"].sort_values(ascending=False)


is_final_girl                 1.000000
number_of_seasons             0.120731
has_reality                   0.120485
is_obscure                    0.112105
is_miniseries                 0.042462
is_youtube                    0.038740
has_documentary               0.032758
is_prime_video                0.026507
number_of_episodes            0.016613
has_amazon_studios            0.007372
popularity                   -0.002215
is_netflix                   -0.006610
is_mid_length                -0.020267
has_universal_television     -0.024271
vote_count                   -0.024719
has_family                   -0.028517
is_long_running              -0.029344
has_warner_bros_television   -0.029703
has_crime                    -0.030167
has_bbc                      -0.036077
is_fox                       -0.036086
is_bbc_one                   -0.051958
is_high_quality              -0.060610
is_hyped                     -0.062159
is_cbs                       -0.062601
is_abc                   

In [28]:
df["scream_queen_score"] = (
    df["is_highly_rated"].astype(int) +
    df["is_popular"].astype(int) +
    df["is_hyped"].astype(int) +
    df["has_drama"].astype(int) +
    df["is_abc"].astype(int) +
    df["has_tvb"].astype(int)
)


In [29]:
df.sort_values("scream_queen_score", ascending=False)[["name", "scream_queen_score", "is_final_girl"]].head(50)


,name,scream_queen_score,is_final_girl
494,Designated Survivor,5,False
276,Station 19,5,True
1562,For Life,5,False
2147,The Time Tunnel,5,False
2217,Grand Hotel,5,False
1445,Resurrection,5,False
1405,Boston Legal,5,False
747,Columbo,5,False
250,The Rookie,5,True
173,Once Upon a Time,5,False


In [30]:
df[df["name"].str.contains("The Walking Dead", case=False)][[
    "name", "is_final_girl", "scream_queen_score", "final_girl_index"
]]


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
df[df["name"].str.contains("The Walking Dead", case=False, na=False)][[
    "name", "is_final_girl", "scream_queen_score", "final_girl_index"
]]

In [ ]:
df["final_girl_index"] = (
    df["is_miniseries"].astype(int) +
    df["is_obscure"].astype(int) +
    df["has_reality"].astype(int) +
    df["is_youtube"].astype(int) +
    df["has_documentary"].astype(int) +
    df["has_amazon_studios"].astype(int)
)


In [ ]:
df[df["name"].str.contains("The Walking Dead", case=False, na=False)][[
    "name", "is_final_girl", "scream_queen_score", "final_girl_index"
]]


In [ ]:
df[df["name"].str.contains("Snowfall", case=False, na=False)][[
    "name", "is_final_girl", "scream_queen_score", "final_girl_index"
]]

In [ ]:
df[df["name"].str.contains("Game of Thrones", case=False, na=False)][[
    "name", "is_final_girl", "scream_queen_score", "final_girl_index"
]]

In [ ]:
df[df["name"].str.contains("Lovecraft Country", case=False, na=False)][[
    "name", "is_final_girl", "scream_queen_score", "final_girl_index"
]]

In [ ]:
df[df["name"].str.contains("The Get Down", case=False, na=False)][[
    "name", "is_final_girl", "scream_queen_score", "final_girl_index"
]]

In [ ]:
# Shows with high final girl index and survived
df[(df["final_girl_index"] >= 3) & (df["is_final_girl"] == True)][[
    "name", "final_girl_index", "scream_queen_score"
]].sort_values("final_girl_index", ascending=False).head(20)


In [ ]:
# Shows with high final girl index and survived
df[(df["final_girl_index"] >= 3) & (df["is_final_girl"] == True)][[
    "name", "final_girl_index", "scream_queen_score"
]].sort_values("final_girl_index", ascending=False).head(100)


In [ ]:
# Shows with high final girl index and survived
df[(df["final_girl_index"] >= 3) & (df["is_final_girl"] == True)][[
    "name", "final_girl_index", "scream_queen_score"
]].sort_values("final_girl_index", ascending=False).head(50)


In [ ]:
df[df["name"].str.contains("The Simpsons", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]


In [ ]:
df[df["name"].str.contains("Bob's Burgers", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]


In [ ]:
df[df["name"].str.contains("Law & Order: Special Victims Unit", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]


### Note: Prestige Final Girls

Some shows survive despite high Scream Queen scores and low Final Girl index. These genre-defying survivors challenge the assumption that niche, low-risk traits are required for survival. This tension reflects the emotional complexity of cancellation and renewal—and reinforces the need for nuanced modeling.


In [ ]:
df.columns


In [ ]:
df[df["name"].str.contains("Scandal", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]


### Scream Queen: Scandal 🩸
Scream Queen Score: 5  
Final Girl Index: 0  
Final Girl Status: ❌ Not a Final Girl  

**Commentary:**  
*Scandal* maxes out the Scream Queen score. Despite its acclaim and cultural impact, it’s the perfect victim. With a Final Girl Index of 0, the model sees no structural protection: no renewal safety net, no low-budget insulation, no survivor coding. A textbook Scream Queen: powerful, exposed, and ultimately unprotected.


In [ ]:
df[df["name"].str.contains("The Get Down", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]

### Scream Queen: The Get Down 🩸 
Scream Queen Score: 4  
Final Girl Index: 0  
Final Girl Status: ❌ Not a Final Girl  

**Commentary:**  
*The Get Down* pulses with Scream Queen energy. It’s the kind of show that burns bright and fast: too ambitious, too expensive, too beautiful to survive. The kind of show that gets killed off before the second act—with a killer soundtrack and no sequel.


In [ ]:
df[df["name"].str.contains("Lovecraft Country", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]

### Scream Queen: Lovecraft Country 🩸 
Scream Queen Score: 4  
Final Girl Index: 0  
Final Girl Status: ❌ Not a Final Girl  

**Commentary:**  
*Lovecraft Country* had a big budget, big monsters, and major bde... big death energy! It walked in an instant hit, a genre-bending fan favorite that got slashed its first season. No sequel. No mercy. Just vibes and a body count.  

In [ ]:
df[df["name"].str.contains("Bob's Burgers", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]

### Final Girl: Bob’s Burgers 🩸 
Scream Queen Score: 3  
Final Girl Index: 0  
Final Girl Status: ✅ Final Girl  

**Commentary:**  
*Bob’s Burgers* is the kind of Final Girl horror nerds never see coming. It’s animated, lives on Fox (aka the network kill zone), and has zero structural protection. The model sees no insulation—just a quirky family, musical numbers, and a renewal miracle. Statistically? It should’ve been a pre-title kill. But somehow, it survives. The kind of show that hides in the walk-in freezer while prestige dramas get gutted in the dining room.


In [ ]:
df[df["name"].str.contains("The Simpsons", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]

### Final Girl: The Simpsons 🩸 
Scream Queen Score: 3  
Final Girl Index: 0  
Final Girl Status: ✅ Final Girl  

**Commentary:**  
*The Simpsons* is the Final Girl that refuses to die. It’s animated, lives on Fox (aka the network kill zone), and scores zero on the Final Girl Index. Statistically? It should’ve been slashed in the ‘90s. But like any good Final Girl, it keeps limping through the decades... bloodied, meta, and somehow still making it to the next episode. The kind of survivor that hides behind a joke and stabs the killer with a rake.


In [ ]:
df[df["name"].str.contains("Law & Order: Special Victims Unit", case=False, na=False)][[
    "name", "is_final_girl", "final_girl_index", "scream_queen_score"
]]

### Final Girl: Law & Order: Special Victims Unit 🩸 
Scream Queen Score: 4  
Final Girl Index: 0  
Final Girl Status: ✅ Final Girl  

**Commentary:**  
*SVU* is the kind of Final Girl that breaks the rules and lives to tell the trauma. The model flags high Scream Queen energy and zero Final Girl Index protection. Procedurals may survive, but crime overall leans slashed: 4.6% slashed rate, 3.3% survival. Statistically? This show should’ve been buried in season six. Instead, it’s still testifying. The kind of survivor that walks into the precinct bloodied, delivers a monologue, and gets renewed before the killer even reloads.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
RANDOM_SEED = 42


In [ ]:
X = df.drop(columns=['is_final_girl'])  # Features
y = df['is_final_girl']                 # Target

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)

In [ ]:
model = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
model.fit(X_train, y_train)

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [ ]:
model = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
model.fit(X_encoded.loc[X_train.index], y_train)

In [ ]:
y_pred = model.predict(X_encoded.loc[X_test.index])
y_prob = model.predict_proba(X_encoded.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))


In [ ]:
y_pred = model.predict(X_encoded.loc[X_test.index])
y_prob = model.predict_proba(X_encoded.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))


In [ ]:
y_pred = model.predict(X_encoded.loc[X_test.index])
y_prob = model.predict_proba(X_encoded.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))
print("Step Eight ran")

In [ ]:
print(len(X_test), len(y_test))

In [ ]:
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
print("X_encoded index:", X_encoded.index[:5])
print("X_test index:", X_test.index[:5])


In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [ ]:
X.select_dtypes(include='object').nunique().sort_values(ascending=False)


In [ ]:
for col in X.select_dtypes(include='object').columns:
    if X[col].apply(lambda x: isinstance(x, list)).any():
        print(f"⚠️ Column '{col}' contains lists")


In [ ]:
for col in ['genre_list', 'network_list', 'company_list']:
    X[col] = X[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


In [ ]:
low_cardinality = X.select_dtypes(include='object').nunique()
print(low_cardinality[low_cardinality < 100].sort_values(ascending=False))


In [ ]:
X_encoded = pd.get_dummies(X, columns=['type', 'status'], drop_first=True)


In [ ]:
model = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
model.fit(X_encoded.loc[X_train.index], y_train)

y_pred = model.predict(X_encoded.loc[X_test.index])
y_prob = model.predict_proba(X_encoded.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))


In [ ]:
X_encoded_clean = X_encoded.select_dtypes(include=['number'])


In [ ]:
model = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
model.fit(X_encoded_clean.loc[X_train.index], y_train)

y_pred = model.predict(X_encoded_clean.loc[X_test.index])
y_prob = model.predict_proba(X_encoded_clean.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))


## 🧪 Baseline Model: Logistic Regression (Selective Encoding)

- Model type: Logistic Regression
- Random seed: 42
- Train/test split: 80/20
- Encoded features: `type`, `status`
- Dropped high-cardinality and list-based columns
- Performance:
  - Accuracy: 72.1%
  - Precision: 67.7%
  - Recall: 63.7%
  - F1 Score: 65.6%
  - AUC: 79.1%

🩸 Commentary: This model is lean, interpretable, and emotionally grounded. It sets the benchmark for survival prediction. The warning signals a pacing issue—not a fatal flaw. We’ll scale and tune later. For now, this is our first scream.


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(model, X_encoded_clean.loc[X_test.index], y_test, cmap='Reds')


## 🔪 Confusion Matrix: Final Girl Baseline

This matrix visualizes how well our model predicts survival:

- **True Positives**: Predicted survivor, actually survived
- **True Negatives**: Predicted non-survivor, actually didn’t survive
- **False Positives**: Predicted survivor, didn’t survive
- **False Negatives**: Predicted non-survivor, actually survived

🩸 Commentary: The matrix reveals where our model hesitates—false negatives are missed final girls, and false positives are false alarms. This visual grounds our intuition and sets the stage for tuning. Every misclassification is a scream we didn’t hear.


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X_encoded_clean),
    index=X_encoded_clean.index,
    columns=X_encoded_clean.columns
)


In [ ]:
model_scaled = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
model_scaled.fit(X_scaled.loc[X_train.index], y_train)

y_pred_scaled = model_scaled.predict(X_scaled.loc[X_test.index])
y_prob_scaled = model_scaled.predict_proba(X_scaled.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred_scaled))
print("Precision:", precision_score(y_test, y_pred_scaled))
print("Recall:", recall_score(y_test, y_pred_scaled))
print("F1 Score:", f1_score(y_test, y_pred_scaled))
print("AUC:", roc_auc_score(y_test, y_prob_scaled))


## 🧪 Scaled Model: Logistic Regression (StandardScaler)

- Model type: Logistic Regression
- Scaling: StandardScaler (mean = 0, std = 1)
- Encoded features: `type`, `status`
- Dropped high-cardinality and list-based columns
- Performance:
  - Accuracy: 73.5% ↑
  - Precision: 71.3% ↑
  - Recall: 61.1% ↓
  - F1 Score: 65.8% ↑
  - AUC: 82.4% ↑

🩸 Commentary: Scaling improved convergence and sharpened the model’s confidence. AUC jumped to 82.4%, signaling stronger separation. This version is emotionally grounded and technically sound—ready for tuning or deployment.


In [ ]:
import joblib

joblib.dump(model_scaled, 'final_girl_logreg_scaled.pkl')


## 💾 Saved Model: final_girl_logreg_scaled.pkl

This file contains the trained logistic regression model using scaled features. It preserves:
- Model weights and coefficients
- Training configuration (`max_iter=1000`, `random_state=42`)
- Feature scaling via `StandardScaler`

🩸 Commentary: This artifact is emotionally grounded and technically reproducible. It marks the end of the baseline arc and the beginning of tuning, interpretation, and deployment.


In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(random_state=RANDOM_SEED)
tree_model.fit(X_scaled.loc[X_train.index], y_train)

y_pred_tree = tree_model.predict(X_scaled.loc[X_test.index])
y_prob_tree = tree_model.predict_proba(X_scaled.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred_tree))
print("Precision:", precision_score(y_test, y_pred_tree))
print("Recall:", recall_score(y_test, y_pred_tree))
print("F1 Score:", f1_score(y_test, y_pred_tree))
print("AUC:", roc_auc_score(y_test, y_prob_tree))


## 🌲 Decision Tree: Ensemble Model

- Model type: DecisionTreeClassifier
- Scaling: StandardScaler
- Performance:
  - Accuracy: 75.0%
  - Precision: 70.3%
  - Recall: 69.7%
  - F1 Score: 70.0%
  - AUC: 74.3%

🩸 Commentary: The Decision Tree is fast and interpretable—ideal for genre-coded survival logic. It improves recall and F1 score over the baseline, but sacrifices a bit of precision and AUC. A solid first ensemble scream.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier(random_state=RANDOM_SEED)
forest_model.fit(X_scaled.loc[X_train.index], y_train)

y_pred_forest = forest_model.predict(X_scaled.loc[X_test.index])
y_prob_forest = forest_model.predict_proba(X_scaled.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred_forest))
print("Precision:", precision_score(y_test, y_pred_forest))
print("Recall:", recall_score(y_test, y_pred_forest))
print("F1 Score:", f1_score(y_test, y_pred_forest))
print("AUC:", roc_auc_score(y_test, y_prob_forest))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier(random_state=RANDOM_SEED)
forest_model.fit(X_scaled.loc[X_train.index], y_train)

y_pred_forest = forest_model.predict(X_scaled.loc[X_test.index])
y_prob_forest = forest_model.predict_proba(X_scaled.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred_forest))
print("Precision:", precision_score(y_test, y_pred_forest))
print("Recall:", recall_score(y_test, y_pred_forest))
print("F1 Score:", f1_score(y_test, y_pred_forest))
print("AUC:", roc_auc_score(y_test, y_prob_forest))


## 🌲 Random Forest: Ensemble Model

- Model type: RandomForestClassifier
- Scaling: StandardScaler
- Performance:
  - Accuracy: 79.2%
  - Precision: 75.9%
  - Recall: 73.4%
  - F1 Score: 74.6%
  - AUC: 88.1%

🩸 Commentary: The Random Forest is your top performer so far—stable, powerful, and emotionally grounded. It balances precision and recall with ensemble clarity, and its AUC leap signals confident separation. A strong contender for the final girl crown.


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

boost_model = GradientBoostingClassifier(random_state=RANDOM_SEED)
boost_model.fit(X_scaled.loc[X_train.index], y_train)

y_pred_boost = boost_model.predict(X_scaled.loc[X_test.index])
y_prob_boost = boost_model.predict_proba(X_scaled.loc[X_test.index])[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred_boost))
print("Precision:", precision_score(y_test, y_pred_boost))
print("Recall:", recall_score(y_test, y_pred_boost))
print("F1 Score:", f1_score(y_test, y_pred_boost))
print("AUC:", roc_auc_score(y_test, y_prob_boost))


## 🔥 Gradient Boosting: Ensemble Model

- Model type: GradientBoostingClassifier
- Scaling: StandardScaler
- Performance:
  - Accuracy: 79.0%
  - Precision: 73.3%
  - Recall: 78.2%
  - F1 Score: 75.7%
  - AUC: 87.6%

🩸 Commentary: Gradient Boosting favors recall and F1, making it emotionally precise and survivor-attuned. It slightly trails Random Forest in AUC but offers a softer, more forgiving margin. A strong contender for emotionally resonant predictions.


## 📊 Ensemble Model Comparison

| Model                | Accuracy | Precision | Recall | F1 Score | AUC   |
|---------------------|----------|-----------|--------|----------|-------|
| Logistic Regression | 75.0%    | 70.3%     | 69.7%  | 70.0%    | 74.3% |
| Decision Tree       | 75.0%    | 70.3%     | 69.7%  | 70.0%    | 74.3% |
| Random Forest       | 79.2%    | 75.9%     | 73.4%  | 74.6%    | 88.1% |
| Gradient Boosting   | 79.0%    | 73.3%     | 78.2%  | 75.7%    | 87.6% |

Random Forest leads in AUC and precision, while Gradient Boosting edges ahead in recall and F1. Both outperform the baseline and tree models, offering technically robust predictions. The final girl crown is Gradient Boosting. It catches more survivors and balances the weight of false negatives. It aligns better with my instincts: softer edges, stronger intuition, and a bias toward protection over perfection.

In [ ]:
import joblib

# Save model and scaler together
joblib.dump({
    'model': boost_model,
    'scaler': scaler
}, 'final_girl_boost_scaled.pkl')


## 💾 Saved Model: final_girl_boost_scaled.pkl

This file contains the trained Gradient Boosting model using scaled features. It preserves:

- Model weights and configuration (random_state=42)
- Feature scaling via StandardScaler

🩸 Commentary: This artifact marks the end of the ensemble arc. It embodies my preference for recall and F1, favoring survivor sensitivity over rigid precision. A deployable scream, ready for interpretation or production.


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0]
}

grid_search = GridSearchCV(
    GradientBoostingClassifier(random_state=RANDOM_SEED),
    param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

grid_search.fit(X_scaled.loc[X_train.index], y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0]
}

grid_search = GridSearchCV(
    GradientBoostingClassifier(random_state=RANDOM_SEED),
    param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1
    verbose=2
)

grid_search.fit(X_scaled.loc[X_train.index], y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

In [ ]:
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)


In [ ]:
try:
    print("Best Parameters:", grid_search.best_params_)
    print("Best F1 Score:", grid_search.best_score_)
except Exception as e:
    print("Error:", e)


In [ ]:
'grid_search' in locals()


In [ ]:
small_grid = {
    'n_estimators': [100],
    'learning_rate': [0.1],
    'max_depth': [3],
    'subsample': [1.0]
}

quick_search = GridSearchCV(
    GradientBoostingClassifier(random_state=RANDOM_SEED),
    small_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

quick_search.fit(X_scaled.loc[X_train.index], y_train)

print("Quick Best Params:", quick_search.best_params_)
print("Quick Best F1 Score:", quick_search.best_score_)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

param_dist = {
    'n_estimators': randint(100, 300),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 6),
    'subsample': uniform(0.7, 0.3)
}

random_search = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=RANDOM_SEED),
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=2,
    random_state=RANDOM_SEED
)

random_search.fit(X_scaled.loc[X_train.index], y_train)

print("Best Parameters:", random_search.best_params_)
print("Best F1 Score:", random_search.best_score_)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
import matplotlib.pyplot as plt

# Confusion Matrix
ConfusionMatrixDisplay.from_estimator(boost_model, X_scaled.loc[X_test.index], y_test)
plt.title("Confusion Matrix: Gradient Boosting")
plt.show()

# ROC Curve
RocCurveDisplay.from_estimator(boost_model, X_scaled.loc[X_test.index], y_test)
plt.title("ROC Curve: Gradient Boosting")
plt.show()


## 🎬 Why This Model Survives

I chose Gradient Boosting as my final girl—not because she’s the loudest, but because she’s the most emotionally attuned. Her recall and F1 score reflect a bias toward protection, catching survivors even when the signal is faint. The ROC curve confirms her sensitivity, and the confusion matrix shows she rarely misfires. She’s not perfect, but she’s precise where it counts. In a genre-coded world, she’s the one who listens, adapts, and endures.